# AI is "cool" with C#/.NET/Azure

<img src=images/jupyter-notebook-meme.webp>

__C# / .NET__
- SIMD; Single instruction, multiple data
    - https://learn.microsoft.com/en-us/dotnet/standard/simd
- Numerics
    - https://learn.microsoft.com/en-us/dotnet/api/system.numerics
- AVX-512 ISA extension; Vector512 and AVX-512
- Parallel Programming in C#/.NET
- Functional Programming features of C#/.NET
    - Async Programming
    - Tuples, Immutable Data Types, Collections
- Dataflow Library
- Generic Math
    - https://learn.microsoft.com/en-us/dotnet/standard/generics/math
- F#
- .NET Interactive

In [ ]:
public static System.Threading.Tasks.Task ForAsync<T>(T fromInclusive, T toExclusive,
    System.Threading.Tasks.ParallelOptions parallelOptions,
    Func<T,System.Threading.CancellationToken,System.Threading.Tasks.ValueTask> body)
    
    where T : System.Numerics.IBinaryInteger<T>;


In [ ]:
#r "System.Runtime.Intrinsics"
System.Runtime.Intrinsics.Vector512.IsHardwareAccelerated

__AI in .NET__
- Data Science libraries
- ML.NET
    - https://www.nuget.org/packages/System.Numerics.Tensors
    - https://www.nuget.org/packages/Microsoft.Data.Analysis
- AutoML / Visual Studio
- Semantic Kernel
- Data Science Tools
    - IronPython
    - Python in Visual Studio and VS Code
    - Polyglot Notebooks

__Microsoft__
- https://github.com/microsoft/CNTK; Microsoft Cognitive Toolkit

*What about PyTorch and TensorFlow?*

__Community__
- https://github.com/dotnet/TorchSharp
- https://scisharp.github.io/SciSharp


__AI in Azure__
- Azure AI Services
- Azure Services that integrates well in AI Workloads; Serverless Computing; Cosmos DB (Notebooks)
- OpenAI Services
- AI Studio

# .NET 💖 AI

- https://dotnet.microsoft.com/en-us/apps/machinelearning-ai

## Overview

<img src=images/ai-net.jpg width=800>

<img src=images/ai-net-2.jpg width=800>

## Microsoft.Data.Analysis

In [ ]:
#r "nuget:Microsoft.Data.Analysis"

In [ ]:
using Microsoft.Data.Analysis;

var series = new PrimitiveDataFrameColumn<int>("MySeries");

series.Append(1);
series.Append(2);
series.Append(3);

Console.WriteLine(series.Length);
Console.WriteLine(series[0]);       // it works like array + something more

series.CumulativeProduct() // pandas has cumprod; multiply value with previous value; 1 as is; 2 will become 2 x 1; 3 will become 3 x 2

- https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.cumprod.html

In [ ]:
// its .NET flavored; implements IEnumerable and hence you can use LINQ
series.OrderByDescending(s => s).Take(1).FirstOrDefault();
// linq and relational queries are good for row level computation
// but its become hard to do reduce like operations where at row operation has some sense of positional context

// in nutshell; you are getting best of both worlds

In [ ]:
using System.IO;
using System.Linq;
using Microsoft.Data.Analysis;

var dataPath = Path.GetFullPath(@"data\house-prices.csv");
var dataFrame = DataFrame.LoadCsv(dataPath);

In [ ]:
//dataFrame.Info()
dataFrame.Description()

In [ ]:
dataFrame["ComputedPrices"] = dataFrame["HistoricalPrice"].Multiply(2);

In [ ]:
dataFrame.GroupBy("Size")

In [ ]:
PrimitiveDataFrameColumn<bool> boolFilter = dataFrame["CurrentPrice"].ElementwiseGreaterThan(200000);
DataFrame filteredDataFrame = dataFrame.Filter(boolFilter);
filteredDataFrame

- https://learn.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/getting-started-dataframe
- https://learn.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/train-machine-learning-model-ml-net

## ML.NET

- https://devblogs.microsoft.com/dotnet/category/ml-dotnet/

### Overview

- https://www.nuget.org/packages/Microsoft.ML
- https://dotnet.microsoft.com/en-us/apps/machinelearning-ai/ml-dotnet
    - https://dotnet.microsoft.com/en-us/learn/ml-dotnet
- https://github.com/dotnet/machinelearning

<img src=images/mlnet-1.png>

<img src=images/mlnet-architecture.png>

In [ ]:
#r "nuget: Microsoft.ML"

In [ ]:
using Microsoft.ML;
using Microsoft.ML.Data;

var mlContext = new MLContext();

In [ ]:
class HouseData
{
    public float Size { get; set; }
    public float Price { get; set; }
}

class Prediction
{
    [ColumnName("Score")]
    public float Price { get; set; }
}

HouseData[] houseData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.4F, Price = 3.7F } };


IDataView trainingData = mlContext.Data.LoadFromEnumerable(houseData);

In [ ]:
var pipeline = mlContext.Transforms.Concatenate("Features", new[] { "Size" })
    .Append(mlContext.Regression.Trainers.Sdca(labelColumnName: "Price", maximumNumberOfIterations: 100));
    
var model = pipeline.Fit(trainingData);

Now that we have the trained model; let test it out

In [ ]:
HouseData[] testData = {
    new HouseData() { Size = 1.1F, Price = 1.2F },
    new HouseData() { Size = 1.2F, Price = 1.5F },
    new HouseData() { Size = 1.4F, Price = 1.7F },
    new HouseData() { Size = 1.6F, Price = 1.9F },
    new HouseData() { Size = 1.9F, Price = 2.3F },
    new HouseData() { Size = 2.8F, Price = 3.0F },
    new HouseData() { Size = 3.2F, Price = 3.5F },
    new HouseData() { Size = 3.3F, Price = 3.6F },
    new HouseData() { Size = 3.5F, Price = 3.9F }, 
    new HouseData() { Size = 3.7F, Price = 4.3F }, 
    new HouseData() { Size = 4.0F, Price = 6.1F }, 
    new HouseData() { Size = 5.0F, Price = 7.2F }, 
    new HouseData() { Size = 6.0F, Price = 8.5F }, 
    new HouseData() { Size = 7.0F, Price = 9.8F }, 
    new HouseData() { Size = 8.0F, Price = 10.3F }, 
};

IDataView trainingTestData = mlContext.Data.LoadFromEnumerable(testData);
IDataView transformedTestData = model.Transform(trainingTestData);

RegressionMetrics trainedModelMetrics = mlContext.Regression.Evaluate(transformedTestData, labelColumnName: "Size");

// Print the R-Squared value. The Closer to 1 indicates a better fitted model.
Console.WriteLine($"Coefficient of determination for the trained model: {trainedModelMetrics.RSquared:0.00}");

If we are happy with the model; we can use this for prediction

In [ ]:
var predictionEngine = mlContext.Model.CreatePredictionEngine<HouseData, Prediction>(model);

var size = new HouseData() { Size = 2.5F };
var price = predictionEngine.Predict(size);
Console.WriteLine($"Predicted price for size: {size.Size*1000} sq ft= {price.Price*100:C}k");

__Resources__
- https://github.com/dotnet/csharp-notebooks/blob/main/machine-learning/01-Intro%20to%20Machine%20Learning.ipynb

<img src=images/mlnet-2.png width=900>

### Yolo

Visual Studio ML.sln / ML.Onnx

<img src=images/yolo.png>

- https://netron.app; open onnx
    - https://github.com/lutzroeder/netron desktop version

__Further Readings__
- https://learn.microsoft.com/en-us/dotnet/machine-learning/how-to-guides/save-load-machine-learning-models-ml-net
- https://learn.microsoft.com/en-us/dotnet/machine-learning/tutorials/object-detection-onnx
    - https://github.com/dotnet/machinelearning-samples/tree/main/samples/csharp/getting-started/DeepLearning_ObjectDetection_Onnx
- https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-automl-onnx-model-dotnet

### Bert

Visual Studio ML.sln / ML.Classifier

- https://blog.google/products/search/search-language-understanding-bert
- https://huggingface.co/docs/transformers/model_doc/bert

<img src=images/bert-embedings.png width=800>

__Further Readings__
- 👉 https://txt.cohere.com/what-are-transformer-models
- https://learn.microsoft.com/en-us/dotnet/machine-learning/tutorials/object-detection-onnx
    - https://github.com/dotnet/machinelearning-samples/tree/main/samples/csharp/getting-started/DeepLearning_ObjectDetection_Onnx
- https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-automl-onnx-model-dotnet

## Model Builder

<img src=images/model-builder-train-done.png width=800>

- https://chat.openai.com/share/a507e145-8d85-4850-b00f-e8c2cb612c56

__Further ML.NET Resources__
- https://learn.microsoft.com/en-us/dotnet/machine-learning/tutorials
- https://github.com/dotnet/machinelearning-samples

## Semantic Kernel

- https://github.com/microsoft/semantic-kernel

<img src=images/sk.png>